# Detective, welcome to the allesfitter software for HackAnExoplanet!
Follow the instructions below to find the best-fit model to the data from a set of selected exoplanet targets.

In [1]:
%%HTML

<style>
    #notebook { padding-top:0px !important; } /* eliminate top gray */
    .container { width:100% !important; }     /* eliminate side gray */
    .end_space { min-height:0px !important; } /* eliminate bottom gray */
</style>

<style>
    div.input { display:none; } /* hide all code cells */
</style>

In [2]:
%%javascript

$('#header').toggle(); // toggle the header off (leaves a grey bar at the bottom)

IPython.OutputArea.prototype._should_scroll = function(lines) { return false; } // disable scrollable output cells

<IPython.core.display.Javascript object>

In [3]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
%matplotlib widget

#::: imports
import os
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML, FileLink
from scipy.interpolate import UnivariateSpline

#::: local imports
import computer
from mcmc import mcmc_fit
from mcmc_output import mcmc_output
from tabulate import tabulate

#::: plotting settings
import seaborn as sns
sns.set(context='paper', style='ticks', palette='deep', font='sans-serif', font_scale=1.5, color_codes=True)
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})
sns.set_context(rc={'lines.markeredgewidth': 1})

#::: globals (don't do this at home, kids)
data = {'time':None, 'flux':None, 'flux_err':None}
model = {'time':None, 'flux':None}
params = {'radius_planet':None, 'radius_star':None, 'epoch':None, 'period':None ,'a':None, 'incl':None}
line = None

In [4]:
###########################################################################
#::: Define all widgets
###########################################################################
#::: widget dropdown, select which data set you want to load
widget_dropdown_targets = widgets.Dropdown(options = ['Select an exoplanet...', 'WASP-189b', 'KELT-3b', 'TOI-560c'], description='Exoplanet:')
#widget_dropdown_targets = widgets.Dropdown(options = ['Select an exoplanet...', 'WASP-189b'], description='Exoplanet:') #minimal version for beta testing

#::: widget sliders, to select the initial guess params
style = {'description_width':'125px'}
layout = {'width':'500px', 'visibility':'hidden'} #hide them to start with
widget_floatslider_radius_planet = widgets.FloatSlider(min=0.1, max=22., value=1., description='Radius of the planet:', style=style, layout=layout)
widget_floatslider_radius_star = widgets.FloatSlider(min=0.1, max=3., value=1., description='Radius of the star:', style=style, layout=layout)
widget_floatslider_epoch = widgets.FloatSlider(min=0., max=1., step=0.01, value=0.3, description='Mid-transit time:', style=style, layout=layout)

#::: widget button, to start the MCMC run
layout = {'visibility':'hidden'} #hide the run button to start with
widget_button_run = widgets.Button(description='Investigate', tooltip='Click here to start the algorithm that fits the model to your data.', layout=layout)

#::: outputs
widget_output0 = widgets.Output(layout={'border':'solid 2px', 'margin':'10px 0px 10px 0px'}) #for the chat; margin 'top/right/bottom/left'
widget_output1 = widgets.Output() #for the light curve
widget_output2 = widgets.Output() #for the histograms
widget_output3 = widgets.Output() #for the table

#::: a tab widget for all fancier output (chat, histograms, table)
# tab = widgets.Tab(children = [widget_output1, widget_output2, widget_output3])
# tab.set_title(0, 'Light curve')
# tab.set_title(1, 'Histograms')
# tab.set_title(2, 'Table')
tab = widgets.Tab(children = [widget_output1])
tab.set_title(0, 'Light curve')

#::: widget images
with open("images/icon1.png", "rb") as file:
    image = file.read()
    im1 = widgets.Image(value=image, format='png', width=150)
with open("images/icon2.png", "rb") as file:
    image = file.read()
    im2 = widgets.Image(value=image, format='png', width=150)
with open("images/icon3.jpg", "rb") as file:
    image = file.read()
    im3 = widgets.Image(value=image, format='jpg', width=150)
    
#::: widget labels
lab1 = widgets.Label(value='Which exoplanet should we look at, Detective?')
lab2 = widgets.Label(value='We need good clues to start digging deeper...')
lab2b = widgets.Label(value='Move the sliders to adjust your initial guess for the model to the data points.')
lab3 = widgets.Label(value='I think we are ready to start investigating, Detective!')
lab_unit1 = widgets.Label(value='Earth radii')
lab_unit2 = widgets.Label(value='Solar radii')
lab_unit3 = widgets.Label(value='days')
lab_units = widgets.VBox(children=[lab_unit1, lab_unit2, lab_unit3])

#::: widget chats
# chat0 = widgets.HTML(value='<style>p{word-wrap: break-word}</style> <p>'+ 
#                             'Good morning, Detective!<br>'+
#                             '</p>',
#                       layout={'width': '330px'})
chat1 = widgets.HTML(value='<style>p{word-wrap: break-word}</style> <p>'+ 
                            'Great work, Detective! Thanks for all the clues. We are running a full investigation now... this will take a few minutes, please be patient.<br>'+
                            'The software is finding the best model that fits the properties of both the host star and the exoplanet, using a mathematical probabilistic technique known as <it>Markov Chain Monte Carlo<\it>.<br>'+
                            'We are also searching in the archives for data from previous observations for some extra clues.'+
                            '</p>')
chat2 = widgets.HTML(value='<style>p{word-wrap: break-word}</style> <p>'+ 
                            'Detective, take a look at this! The investigation was successful, and now we have a better understanding of what happened.<br>'+
                            'We also found some old case files in the archive that have given us extra insight.<br>'+
                            'We have prepared a case file for you with all the details - have a look: '+
                            '</p>')
# with widget_output0:
#     display(chat0)
    

    
###########################################################################
#::: Clean up and set up the new figure
###########################################################################
plt.close('all')
with widget_output1:
    fig, ax = plt.subplots(1, figsize=(12, 6), tight_layout=True)
    plt.show(fig)
widget_output0.layout.visibility = 'hidden'
tab.layout.visibility = 'hidden'



###########################################################################
#::: Define all interactive actions
###########################################################################
#::: define the function to execute after the selection
def load_and_plot_data(target_name):
    
    #::: catching the blank state
    if target_name == 'Select an exoplanet...':
        
        #::: handle the figure
        ax.clear()
        #ax.set(xlabel='Time (Days)', ylabel='Light from star (%)', title='No exoplanet selected')
        #text = ax.text(0.5, 0.5, '?', fontsize=64)
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
            
        #::: temporarily hide the sliders and button
        im2.layout.visibility = 'hidden'
        lab2.layout.visibility = 'hidden'
        lab2b.layout.visibility = 'hidden'
        lab_units.layout.visibility = 'hidden'
        widget_floatslider_radius_planet.layout.visibility = 'hidden'
        widget_floatslider_radius_star.layout.visibility = 'hidden'
        widget_floatslider_epoch.layout.visibility = 'hidden'
        im3.layout.visibility = 'hidden'
        lab3.layout.visibility = 'hidden'
        widget_button_run.layout.visibility = 'hidden'
        widget_output0.layout.visibility = 'hidden'
        tab.layout.visibility = 'hidden'
        
        #::: end here
        return None
    
    else:
        #::: unhide the sliders (but keep the button hidden)
        ax.xaxis.set_visible(True)
        ax.yaxis.set_visible(True)
        im2.layout.visibility = 'visible'
        lab2.layout.visibility = 'visible'
        lab2b.layout.visibility = 'visible'
        lab_units.layout.visibility = 'visible'
        widget_floatslider_radius_planet.layout.visibility = 'visible'
        widget_floatslider_radius_star.layout.visibility = 'visible'
        widget_floatslider_epoch.layout.visibility = 'visible'
        tab.layout.visibility = 'visible'
    
    #::: globals (don't do this at home, kids)
    global data
    global model
    global params
    global line
    
    #::: load data
    data['time'], data['flux'], data['flux_err'], model['time'] = computer.load_data(target_name)
    
    #::: load frozen params and initial guesses for the sliders
    params['radius_planet'], params['radius_star'], params['epoch'], params['period'], params['a'], params['incl'] = computer.load_params(target_name)
    
    #::: set the sliders #TODO fix this: somehow this does not reset all slides when switching targets, but only the first one that has not already been reset...
    widget_floatslider_radius_planet.value = params['radius_planet']
    widget_floatslider_radius_star.value = params['radius_star']
    widget_floatslider_epoch.value = params['epoch']
    
    #::: plot the data and an empty model
    with widget_output1:
        ax.clear()
        ax.set(xlabel='Time (Days)', ylabel='Light from star (%)', title='Transit Light Curve of '+target_name)
        ax.plot(data['time'], 100.*data['flux'], 'b.', label='Data points')
        line, = ax.plot(model['time'], 100.*np.ones_like(model['time']), color='silver', marker=None, linestyle='-', label='Model based on your first clues')
        ax.legend(loc='lower left')
        fig.canvas.draw_idle() #this change was pivotal to make the interactive plot work


    
#::: update the plot with the light curve model
def update_plot(radius_planet, radius_star, epoch):
    
    #::: catching the blank state
    if widget_dropdown_targets.value == 'Select an exoplanet...':
        return None #abort
    
    #::: globals (don't do this at home, kids)
    global data
    global model
    global params
    global line
    
    #::: set params
    params['radius_planet'] = radius_planet
    params['radius_star'] = radius_star
    params['epoch'] = epoch
    
    #::: retrieve the light curve model
    model['flux'] = computer.calc_flux_model(params['radius_planet'], params['radius_star'], params['epoch'], params['period'], params['a'], params['incl'], model['time'])
    
    #::: update the axes (try/except catches some initialisation hickups)
    try:
        line.set_ydata(100.*model['flux'])
    except:
        pass
    with widget_output0:
        fig.canvas.draw_idle() #this change was pivotal to make the interactive plot work
    
    #::: unhide/hide the button, depending on how close the initial guess is to the truth
    if computer.check_initial_guess(widget_dropdown_targets.value, params['radius_planet'], params['radius_star'], params['epoch']):
        im3.layout.visibility = 'visible'
        lab3.layout.visibility = 'visible'
        widget_button_run.layout.visibility = 'visible'
    else:
        im3.layout.visibility = 'hidden'
        lab3.layout.visibility = 'hidden'
        widget_button_run.layout.visibility = 'hidden'
    
    

#::: define the run button
def run(arg):
    
    #::: disable all widgets for luser-proofness
    widget_button_run.disabled = True
    widget_dropdown_targets.disabled = True
    widget_floatslider_radius_planet.disabled = True
    widget_floatslider_radius_star.disabled = True
    widget_floatslider_epoch.disabled = True
    
    #::: make the chat visible
    widget_output0.layout.visibility = 'visible'
    
    #::: change the line color of the initial guess
    #line.set_color('silver') #TODO: somehow this only takes effect after the MCMC run is completed
    
    #::: print statements and other output need an "output widget" (figures do not)
    with widget_output0:
         
        #::: start the MCMC run 
        display(chat1)
        mcmc_fit(widget_dropdown_targets.value, params)
        
        #::: get the MCMC output
        display(chat2)
        posterior_samples, fig_hist, table = mcmc_output(widget_dropdown_targets.value, params=params) #20 samples for plotting
        #plt.close(fig_hist) #for now
        table = tabulate(table, tablefmt='html', headers=['Name', 'Median value', 'Lower error', 'Upper error', 'Case note', 'Target']) #for now
        

    #::: plot the fit
    for i in range(20):

        #::: compute the ellc model (on finer time grid)
        model['flux'] = computer.calc_flux_model(posterior_samples['radius_planet'][i], posterior_samples['radius_star'][i], posterior_samples['epoch'][i], params['period'], params['a'], params['incl'], model['time'])

        #::: compute the the baseline
        y2 = computer.calc_flux_model(posterior_samples['radius_planet'][i], posterior_samples['radius_star'][i], posterior_samples['epoch'][i], params['period'], params['a'], params['incl'], data['time']) #get model on data time grid
        yerr_weights = data['flux_err']/np.nanmean(data['flux_err'])
        weights = 1./yerr_weights
        spl = UnivariateSpline(data['time'], data['flux']-y2, w=weights, s=np.sum(weights)) #train a spline on the data time grid
        baseline = spl(model['time']) #evaluate the spline on the finer time grid

        #::: plot
        ax.plot(model['time'], 100.*model['flux']+baseline, 'r-', alpha=0.1)

    #::: add a legend
    ax.plot(np.NaN, np.NaN, 'r-', label='Final results of your investigation')
    ax.legend(loc='lower left') 

    #::: save the light curve figure
    fig.savefig( os.path.join('results',widget_dropdown_targets.value,'light_curve.pdf'), bbox_inches='tight' )

    
    #::: add hist and table children to the tab widget
    tab.children = [widget_output1, widget_output2, widget_output3]
    tab.set_title(0, 'Light curve')
    tab.set_title(1, 'Histograms')
    tab.set_title(2, 'Table')
    

    #::: print statements and other output need an "output widget" (figures do not)
    with widget_output1:
        
        #::: show the file links
        #::: not really needed for mybinder's jupyter notebook, as figures show up there by default (but needed for jupyter lab)
        # plt.show(fig_hist) #not needed for mybinder's jupyter notebook, as figures show up there by default (but needed for jupyter lab)
        #txt1 = 'Download the light curve figure here:'
        txt1 = widgets.Label(value='Download the light curve figure here:')
        fname1 = os.path.join('results',widget_dropdown_targets.value,'light_curve.pdf')
        display(txt1, FileLink(fname1))
    
    with widget_output2:
        plt.show(fig_hist)
        txt2a = widgets.Label(value='The histograms show the probability of each parameter having a certain value.')
        txt2b = widgets.Label(value='The central, solid line shows the median value of each paramter.')
        txt2c = widgets.Label(value='The dashed lines to the left and right of it indicate the lower and upper bounds, respectively.')
        txt2d = widgets.Label(value='These are called the 1-sigma uncertainties. That means, statistically we can be 68% sure that the true value lies within them.')
        txt2e = widgets.Label(value='Note that this means it is possible that the true value of a parameters lies outside of these bounds;')
        txt2f = widgets.Label(value='they are only statistical uncertainties, not definite limits.')
        txt2g = widgets.Label(value='Download the histogram figure here:')
        box_txt2 = widgets.VBox(children=[txt2a, txt2b, txt2c, txt2d, txt2e, txt2f, txt2g])
        fname2 = os.path.join('results',widget_dropdown_targets.value,'histograms.pdf')
        display(box_txt2, FileLink(fname2))
        
    with widget_output3:
        display(table)
        txt3 = widgets.Label(value='Download the table here:')
        fname3 = os.path.join('results',widget_dropdown_targets.value,'table.txt')
        display(txt3, FileLink(fname3))
    
    

#::: link the dropdown menu to the creation of the plot
w1 = widgets.interactive(load_and_plot_data, 
                         target_name=widget_dropdown_targets)



#::: now let's interact with this plot
w2 = widgets.interactive(update_plot,
                         radius_planet = widget_floatslider_radius_planet, 
                         radius_star = widget_floatslider_radius_star, 
                         epoch = widget_floatslider_epoch)



#::: execute clicks on the run button
widget_button_run.on_click(run)



###########################################################################
#::: Display everything neatly
###########################################################################
#::: display all widgets in a box
box1 = widgets.VBox(children=[im1, lab1, w1], layout={'align_items':'center'})
box2a = widgets.HBox(children=[w2, lab_units])
box2 = widgets.VBox(children=[im2, lab2, box2a, lab2b], layout={'align_items':'center', 'width':'750px'}) #set this width to establish some space
box3 = widgets.VBox(children=[im3, lab3, widget_button_run], layout={'align_items':'center'})

box4 = widgets.HBox(children=[box1, box2, box3], layout={'border':'solid 2px'})
box5 = widgets.VBox(children=[box4, widget_output0, tab])
# box = widgets.VBox(childred=[box1, box2]) #does not work for some reason

display(box5) # <- this one command displays all children